In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from pyspark.sql.session import SparkSession
from pyspark.sql.context import SQLContext

In [2]:
user = "postgres"
password = "Felipe1998"
hostname = "localhost"
db_name = "postgres"

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{hostname}/{db_name}")

In [3]:
create_table_query ="""

    DROP TABLE IF EXISTS pyspark_tables.fire_service_calls CASCADE;

    CREATE TABLE pyspark_tables.fire_service_calls(
        call_number INTEGER,
        unit_id TEXT,
        incident_number INTEGER NOT NULL,
        call_type TEXT,
        call_date TIMESTAMP,
        watch_date TIMESTAMP,
        received_dttm TIMESTAMP,
        entry_dttm TIMESTAMP,
        dispatch_dttm TIMESTAMP,
        response_dttm TIMESTAMP,
        on_scene_dttm TIMESTAMP,
        transport_dttm TIMESTAMP,
        hospital_dttm TIMESTAMP,
        call_final_disposition TEXT,
        available_dttm TIMESTAMP,
        address TEXT,
        city TEXT,
        zipcode_of_incident INTEGER,
        battalion TEXT,
        station_area INTEGER,
        box INTEGER,
        original_priority TEXT,
        priority TEXT,
        final_priority INTEGER,
        als_unit BOOLEAN,
        call_type_group TEXT,
        number_of_alarms INTEGER,
        unit_type TEXT,
        unit_sequence_in_call_dispatch INTEGER,
        fire_prevention_district TEXT,
        supervisor_district TEXT,
        neighborhoods_analysis_boundaries TEXT,
        rowid TEXT,
        case_location TEXT,
        data_as_of TIMESTAMP,
        data_loaded_at TIMESTAMP,
        computed_region_ajp5_b2md INTEGER,

        PRIMARY KEY(incident_number)
    );

"""

engine.execute(create_table_query)

In [4]:
#spark session
spark = SparkSession.builder.appName("CSV_todatabase")\
                            .config("spark.driver.extraClassPath","../helpers/postgresql-42.7.3.jar")\
                            .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0")\
                            .getOrCreate()
#sql context
sqlcontext = SQLContext(spark)

c:\Users\juanf\anaconda3\envs\dataenv\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
#reading a csv file.
sf_fire_calls_df = spark.read.csv("../data/fire_department_calls.csv", header=True, inferSchema=True)

In [6]:
#defining the jdbc connection
jdbc_url = "jdbc:postgresql://localhost:5432/postgres"

In [7]:
#write to SQL
sf_fire_calls_df.write.format("jdbc")\
                      .option("url",f"{jdbc_url}")\
                      .option("driver", "org.postgresql.Driver")\
                      .option("user","postgres")\
                      .option("password","Felipe1998")\
                      .option("dbtable","pyspark_tables.fire_service_calls")\
                      .mode("overwrite").save()